In [ ]:
// general includes
#include <iostream>   // std::cout|endl 
#include <vector>   // std::vector
// using namespace std;
struct NoWidget {};
struct WidgetM {
    int m;
};

Hallo

## Quick Recap: special member functions (SMFs)

- when are they available implicitly?
- how many are there?
- should the user always implement all of them by hand for user-defined classes?

  ```cpp
  struct Widget {
    double *ptr; // dynamic resource owned by Widget
    // SMFs?
  };
  ```
- "rule of 3": resource owning classes should implement 3 SMFs: copy constructor/copy assignment/destructor
- "rule of 5": extend for move semantics (always optional): move constructor/move assignment
- Any owning class: do rule 3 / 5
- "rule of 0": if the class does not own resources "directly" --> rely on implicit SMFs


## Goals for today
- Understand the concept of function templates
- Understand the close relation of function templates and `auto`
  - `auto` is great

# Function templates

In C++ has *templates* [(cppref)](https://en.cppreference.com/w/cpp/language/templates) which allow to implement generic functionality. We will look closer at *function templates* [(cppref)](https://en.cppreference.com/w/cpp/language/function_template).

Templates are might but often hard to deal with

Let's assume we want to implement a non-member function `swap` which exchanges the contents of two objects of equal type `Widget`:

In [7]:
struct Widget {
  int m;
};

What do you do, if you have hundreds of classes

If the lines are same, but only the class changes, then use TEMPLATES !

In [8]:
void swap(Widget &a, Widget &b) { // takes two references, should swap 
  Widget tmp(std::move(a));     // save tmp (1. move) // (1) which SMF is used? move ctor
  a = std::move(b);             // 2. move            // (2) which SMF is used? move assignment
  b = std::move(tmp);           // 3. move            // (3) which SMF is used? move assignment
};

**What requirements are there towards the `Widget` class?**
- move ctor (implicitly available?, yes)
- move assign (implicitly available? yes)
- ... copy versions of ctor and assign are also sufficient as a fallback

To make this swap functionality available for other types too, overloads could be used:

In [9]:
struct Type1 {};
struct Type2 {};
struct Type3 {};

In [10]:
void swap(Type1 &a, Type1 &b) { }; // for Type1/Type1

In [11]:
void swap(Type2 &a, Type2 &b) { }; // for Type2/Type2

In [12]:
void swap(Type3 &a, Type3 &b) { }; // for Type3/Type3

**Is the "overload approach" good practice if the number of types to be supported is large?**
- if a set of types you want to support needs exactly the same implementation then a template is a good idea
- if a set of types each needs special treatment: work with overloads

Let's look at a situation where the implementation (the expressions in the function body) of `swap` does not depend on the type of the parameters. We can illustrate this by using a *type alias* which highlights the influence of the type of the parameters on the function body:

In [13]:
using T = Type1; // only changing this type alias for Type2, Type3, Type4

In [14]:
void swap(T& a, T& b) { //it alreasy looks like a template but it is not
  T tmp(std::move(a));
  a = std::move(b);
  b = std::move(tmp);
};

The syntax to implementing the above `swap` as a *function template* [(cppref)](https://en.cppreference.com/w/cpp/language/function_template) with a single *type template parameter* [(cppref)](https://en.cppreference.com/w/cpp/language/template_parameters) `T` looks like this:

In [15]:
template <typename T >  // template function with a "type template parameter" 'T' 
void swap(T& a, T& b)  { //this is now template. Language automattically detects template type in code then
  //theoretically works for all classes
  //but of course does not in reality
  //lately you can restrict Type that CAN be used for this template (C++ 20)
  //Breaks on compile time
  T tmp(std::move(a));
  a = std::move(b);
  b = std::move(tmp);
}; 

**What are the prerequisites for type `T` to be "compatible" with this function template?**
- move-constructable (or fallback to copy)
- move-assignable (or fallback to copy)

The definition of a template (function) is literally a template for the compiler:
- If the templated function **is not selected** to be used (anywhere in the translation unit), no code is generated for the (not selected) function.
- If the templated function **is selected** to be used, code is generated by the compiler (*instantiation*) - potentially multiple times for different types - according to the template definition.

## Template argument determination
In order to instantiate (or try to instantiate) any template, all arguments for the template parameters must be known to the compiler. 
There are three mechanism which lead to the determination of function template parameters (precedence in this order):

- explicitly specifying the arguments 

In [18]:
 Widget a{1};
 Widget b{2};
 //swap<Widget>(a, b); // instantiates void swap<Widget>(Widget&, Widget&)

 //try
 //std::string mystr = "aaaa";
 //swap<Widget>(a,b); //would not work

 //you can do it like
 template <typename T1, typename T2>
 void swap(T1& a, T2& b){}

input_line_28:9:2: error: expected expression
 template <typename T1, typename T2>
 ^


Interpreter Error: 

- deduction from function arguments 

In [15]:
Widget a{1};
Widget b{2}; // different type
swap(a,b);         // instantiates void swap<Widget>(Widget&, Widget&)

- default arguments for template parameters are defined 

In [18]:
template <typename T = Widget> 
void swap(T& a, T& b) {
    T tmp(std::move(a));
    a = std::move(b);
    b = std::move(tmp);
};

In [19]:
swap(a,b);   // instantiates void swap<Widget>(Widget&, Widget&) 

- one more example for default arguments combined with "relations" between template parameters:

In [23]:
template <typename T, typename D = decltype(T().m)> 
void swapM(T& a, T& b) {
    D tmp(std::move(a.m));
    a.m = std::move(b.m);
    b.m = std::move(tmp);
};

In [25]:
WidgetM a{1};
WidgetM b{2};  
swapM<WidgetM>(a,b);   // instantiates void swapM<Widget,int>(Widget&, Widget&)

**Could the last snippet also have been implemented without the extra template parameter `D`?**
- yes, using the `auto` type placeholder.

## Function template argument deduction
Let's look closer at the rules for deducing template arguments from function arguments [(cppref)](https://en.cppreference.com/w/cpp/language/function_template#Template_argument_deduction).
We will consider a function template with a single argument of a form similar to this:

In [26]:
template <typename AUTO> 
void func(AUTO arg) {};

The type of the function parameter is marked in red and the template parameter is is marked in bold blue.
A type is deduced for both, influenced by the expression passed as an argument:


In [28]:
double expr = 1.0;
func(expr); //internally: auto arg = (expr);

**Note:** The rules for template argument deduction are also applied when using `auto` as a placeholder type for variables, i.e., the following snippet will use the same mechanism to deduce the type for `arg` from the expression `expr` as the function template above. 

In [29]:
auto arg = (expr); //when you do that, you the same thing as above

In the following we will consider these four different scenarios for deduction:

In [30]:
template <typename AUTO> 
void func1(AUTO arg) {}; // pass-by-value
//we get a copy inside the func
//do not know ehther it is const etc

In [3]:
template <typename AUTO> 
void func2(AUTO& arg) {}; // pass-by-reference. I want an Lvalue reference

auto& dd = (expr);

//What is lost here: cannot have passing by value
//cannot move on position


input_line_11:3:13: error: use of undeclared identifier 'expr'
auto& dd = (expr);
            ^


Interpreter Error: 

In [32]:
template <typename AUTO> 
void func3(const AUTO& arg) {}; // pass-by-reference-to-const

const auto& ddd = (expr);

In [4]:
template <typename AUTO> 
void func4(AUTO&& arg) {}; // pass-by-forwarding-references. SPECIAL!!! 

const auto&& dddd = (expr);
//you do not know what was in the outside

input_line_12:3:22: error: use of undeclared identifier 'expr'
const auto&& dddd = (expr);
                     ^


Interpreter Error: 

### Pass-by-value
If the function parameter is passed by-value the function template looks like this:

In [36]:
template <typename AUTO> 
void func1(AUTO arg) {}; // pass-by-value

In [37]:
Widget lval{1};    
Widget &lref = lval;   
const Widget &clref = lval;  
Widget *ptr = &lval; 
const Widget *cptr = &lval; 
const Widget *const cptrc = &lval; 

func1(lval);                 // func<Widget>(Widget arg) for lvalue
func1(lref);                 // func<Widget>(Widget arg) for lvalue reference
func1(clref);                // func<Widget>(Widget arg) for lvalue reference to const
func1(Widget{});             // func<Widget>(Widget arg) for rvalue
func1(std::move(lval));      // func<Widget>(Widget arg) for rvalue reference
func1(std::move(clref));     // func<Widget>(Widget arg) for rvalue reference to const    
func1(ptr);                  // func<Widget *>(Widget *arg) for ptr
func1(cptr);                 // func<const Widget *>(const Widget *arg) for ptr to const 
func1(cptrc);                // func<const Widget *>(const Widget *arg) for const ptr to const

To summarize deduction for a pass-by-value function parameter: 
- `const` is dropped 
- for pointers const access to the pointee is not dropped
- the two deduced types (template parameter type and function parameter type) are identical.

**Do these rules for pass-by-value make sense?**
- const-ness is not preserved(dropped): this makes sense, as everything is pass-by-value (copied)
- reference-ness is not preserved: makes sense, we asked for a copy
- pointers: passing a pointer-to-const preserves the const-ness; this is what is expected from the caller site.

### Pass-by-reference
If the function parameter is *passed by-reference* the function template looks like this:

In [39]:
template <typename AUTO> 
void func2(AUTO& arg) {}; // pass-by-reference

The following snippet again illustrates the deduction for different value categories and types:


In [40]:
Widget lval{1};    
Widget &lref = lval;   
const Widget &clref = lval;  
Widget *ptr = &lval; 
const Widget *cptr = &lval; 
const Widget *const ptrc = &lval; 
func2(lval);                 // func<Widget>(Widget &arg) for lvalue
func2(lref);                 // func<Widget>(Widget &arg) for lvalue reference
func2(clref);                // func<const Widget>(const Widget &arg) for lvalue reference to const   
func2(ptr);                  // func<Widget *>(Widget *&arg) for ptr
func2(cptr);                 // func<const Widget *>(const Widget *&arg) for ptr to const
func2(cptrc);                // func<const Widget *const>(const Widget *const &arg) for const ptr to const

To summarize for a pass-by-reference function parameter: 
- `const`-ness is preserved in both deduced types
- if `expr` is a reference type or not does not influence the deduction
- function parameter type is always a lvalue reference
- the template parameter is never of reference type


**Do these pass-by-ref rules make sense?**
- const-ness is preserved: this is what is expected when passing a const or a const reference
- const-ness is also preserved in the template parameter type: this allows to specialize the implementation if a function parameter is const
- the function parameter type is always a lvalue reference: this is what we asked for

### Pass-by-reference-to-const 
If the function parameter is passed by a lvalue-reference to const, the function template looks like this:

In [42]:
template <typename AUTO> 
//cinst reference
void func3(const AUTO& arg) {}; // pass-by-reference-to-const

And again, following snippet illustrates the deduction for different value categories and types:

In [43]:
Widget lval{1};    
Widget &lref = lval;   
const Widget &clref = lval;  
Widget *ptr = &lval; 
const Widget *cptr = &lval; 
const Widget *const cptrc = &lval; 
func3(lval);                 // func<Widget>(const Widget &arg) for lvalue/reference
func3(clref);                // func<Widget>(const Widget &arg) for lvalue/reference to const   
func3(Widget{});             // func<Widget>(const Widget &arg) for rvalue/reference  

func3(ptr);                  // func<Widget *>(Widget *const &arg) for ptr
//The POINTER is set const, NOT the ref! (thing itself)
func3(cptr);                 // func<const Widget *>(const Widget *const &arg) for ptr to const
func3(cptrc);                // func<const Widget *>(const Widget *const &arg) for const ptr to const

To summarize for a pass-by-reference-to-const function parameter: 
- the "forced" `const`-ness is not reflected in the type of the template parameter
- function parameter type is always a const lvalue reference
- the template parameter is never of reference type


**Do these rules for pass-by-const-ref make sense?**
- "original" const-ness is lost: this is what we asked for: to always bind to an lvalue reference to const
- pointer: const is "forced" for the pointer type itself, e.g., `Widget *const`: this is expected; see below snippet for a  syntax which reveals this:
  ```cpp
    using Pointer = Widget *; // equivalent: Widget *
    using ConstPointer = const Pointer; // equivalent: Widget *const
    //const applied to pointer, not underlying type
    using PointerToConst = const Widget *; // equivalent: const Widget *
    using ConstPointerToConst = const PointerToConst; // equivalent: const Widget *const
  ```

**Is any of the template deduction mechanism above suitable to maintain the full information of an argument (i.e., value-category, const-ness, and type), and why would this even be important?**
- No:
  - pass-by-value hides the reference-ness (and forces parameters to be constructible)
  - pass-by reference (non-const) cannot bind to rvalues
  - pass-by const reference can bind to anything via a reference, but also marks everything const. 
- Often, a "perfect forwarding" of function parameters is highly desirable, e.g., passing the parameters to a nested function without the nested function experiencing any difference (compared to a direct invocation). 
- Forwarding references (see below) combined with the reference collapsing can achieve this.

In [19]:
template <typename AUTO> 
void func1(AUTO arg) { /* no info if arg was e.g., movable*/ };
template <typename AUTO> 
void func2(AUTO& arg) { /* only binds to lvalues, rvalues do not work */};
template <typename AUTO> 
void func3(const AUTO& arg) { /* const is forced, no info if actually was const */ };

### Pass-by-forwarding-reference
In the semantic embedding of a function template the syntax for a non-const rvalue reference has a special meaning:


In [20]:
template <typename AUTO> 
void func4(AUTO&& arg) {}; // pass-by-forwarding-references

Here `&&` denotes a so-called *forwarding reference* [(cppref)](https://en.cppreference.com/w/cpp/language/reference#Forwarding_references) (or also *universal reference*) with special deduction rules.
The following snippet illustrates the deduction when using a forwarding reference:

In [21]:
Widget lval{1};    
Widget &lref = lval;   
const Widget &clref = lval;  
Widget *ptr = &lval; 
const Widget *cptr = &lval; 
const Widget *const cptrc = &lval; 
func4(lval);                 // func<Widget &>(Widget &arg) for lvalue
func4(lref);                 // func<Widget &>(Widget &arg) for lvalue reference
func4(clref);                // func<const Widget &>(const Widget &arg) for lvalue reference to const
func4(Widget{});             // func<Widget>(Widget &&arg) for rvalue
func4(std::move(lval));      // func<Widget>(Widget &&arg) for rvalue reference
func4(std::move(clref));     // func<const Widget>(const Widget &&arg) for rvalue reference to const    
func4(ptr);                  // func<Widget *&>(Widget *&arg) for ptr
func4(cptr);                 // func<const Widget *&>(const Widget *&arg) for ptr to const
func4(cptrc);                // func<const Widget *const &>(const Widget *const &arg) for const ptr to const

To summarize when using a forwarding reference as function parameter: 
- the const-ness and value-ness of the expression passed to the function are fully preserved in the type of the function parameter
- the template parameter is a reference type if the expression was an lvalue; for rvalue it is non-reference type


**Do these rules for forwarding-refs make sense?**
- the rules aim to achieve a specific goal: to preserve the value category and const-ness of the originally passed expression (to be able to forward it later).
- the template parameter is non-reference for rvalues: this is OK (instead of being of rvalue reference) as it has no effect on reference collapsing [(cppref)](https://en.cppreference.com/w/cpp/language/reference) (which applied during forwarding)

**Note**:
This preservation of the "original" value category and const-ness in the type of the function parameters sets the stage for "perfect forwarding" of parameters to nested functions. To achieve this, additionally the "reference collapsing rules" and `std::forward` (which makes use of these rules) are required.

## Template parameter pack 
C++ allows to specify a *template parameter pack* [(cppref)](https://en.cppreference.com/w/cpp/language/parameter_pack) (e.g., `typename ...ARGS`) as template parameter which can be mapped to a *function parameter pack* (e.g., `func(ARGS&&...args)`).
This allows to construct function templates accepting a variable number of arguments:

In [22]:
template<typename ...ARGS> // named template parameter pack
void function(ARGS... args) { // function parameter pack
  //...  how to access/deal with args here?
}

Let's look at some examples to illustrate some use cases using *fold expressions* [(cppref)](https://en.cppreference.com/w/cpp/language/fold):

In [23]:
template <typename... ARGS> auto sum_args(ARGS &&... args) {
  return (0 + ... + args); // sums all args, initial value '0'
};

input_line_33:2:15: warning: pack fold expression is a C++17 extension [-Wc++17-extensions]
  return (0 + ... + args); // sums all args, initial value '0'
              ^


In [24]:
template <typename... ARGS> auto print_args(ARGS &&... args) {
  ((std::cout << args << ','), ..., (std::cout << std::endl)); // prints all args
};

input_line_34:2:32: warning: pack fold expression is a C++17 extension [-Wc++17-extensions]
  ((std::cout << args << ','), ..., (std::cout << std::endl)); // prints all args
                               ^


Can we also use this to wrap a function?

In [54]:
template <typename... ARGS> auto forward_args(ARGS &&... args) {
  return func(args...); // passes all args to 'func' in same order
};

**Why is the last above snippet not "perfect forwarding"?**
- rvalue references with a name will be received as lvalues when binding to the parameters of `func`
- to forward correctly, rvalue arguments (but only those) have to "moved", this is achieved by `std::forward` [(cppref)](https://en.cppreference.com/w/cpp/utility/forward):

In [27]:
  template <typename... ARGS> auto perfect_forward_args(ARGS &&... args) {
    //ARGS can be everything
    //only movables will be moved.
    return func(std::forward<ARGS>(args)...);
  };

**What does `std::forward` do?**
- it performs a cast to an rvalue (rvalue reference), quite similar to `std::move`
- the difference is that it acts selectively (by relying on the rules of reference collapsing).

A parameter pack can also be combine with "regular" template parameters. Below this is used to build a "perfect" function wrapper:

CALLABLE: not even bound to the function
    - Synthax : it has to be callable ( `operator()` ), and defined with the passsed parameter.

In [28]:
template <typename CALLABLE, typename... ARGS>
auto perfect_function_wrapper(CALLABLE callable , ARGS &&... args) {
  return callable(std::forward<ARGS>(args)...); 
};

**What are the requirements for the `callable` in above snippet?**
- `operator()` with a fitting sequence of parameters (i.e. which can bind to the sequence of `args`)

## `auto` placeholder type [(cppref)](https://en.cppreference.com/w/cpp/language/auto)

The deduction rules for function template arguments are also used when `auto` is used as a placeholder for the type of a variable:

In [63]:
double expr = 1.0;
auto arg1 = (expr); // (1) non-reference type

auto& arg2 = (expr); // (2) lvalue reference type

// the next two "always bind"
const auto& arg3 = (expr); // (3) const lvalue reference type
auto&&  arg4 = (expr); // (4) forwarding reference!

**How do the analog function templates for the expressions above look like?**

An very illustrative example of the use of `auto` and its consequences it a range-based for loop:
- What is there are heavy items in there?


In [64]:
std::vector<double> list = {1,2,3};

for (auto item: list) {
  // ...
}

for (auto& item: list) {
  // ...
}

for (const auto& item: list) {
  // ...
}


**Discuss the consequences of the above choices for the loop!**

**Is this really exactly the same mechanism for function templates and `auto` type placeholders?**
- yes, but one exception concerning `std::initializer_list` [(cppref)](https://en.cppreference.com/w/cpp/utility/initializer_list):
  ```cpp
  // example for only difference: std::initializer_list
  template <typename AUTO>
  void func(std::initializer_list<AUTO> arg){};

  int main() {
    auto arg = {1, 2, 3}; // works
    func({1, 2, 3});      // does not work
  }
  ```

In [ ]:
const int N = 4096;
float A[N*N]; //64MB: out of stack
//out of range of mem to capture memory


float A = new float[N*N];
delete[] A;

# Summary

- Function templates
- `auto`
- Wrapping is very desirable, can do with forward ref + `forward`